In [47]:
#import libraries and functions
import numpy as np
from arqtic.program import Program, random_bitstring
from arqtic.hamiltonians import Ising_Hamiltonian
from arqtic.arqtic_for_ibm import run_ibm
import arqtic.QITE as qite
import qiskit as qk
from qiskit import Aer, IBMQ, execute

In [53]:
#interfacing with IBM
########################################
#The following only needs to be run if this you very first time running this pynb, comment out all subsequent times
#qk.IBMQ.save_account('API_token_here')
#qk.IBMQ.load_account()
########################################

#set up simulator
simulator = Aer.get_backend('qasm_simulator')

In [48]:
#define post-processing functions
#get amouot of work from QC results
def get_work(results, shots):
    work = 0
    for result in results:
        state_vec = result[0]
        count = result[1]
        work_comp = 0
        for i in range(len(state_vec)):
            if (state_vec[i] == 0):
                work_comp += 1
            else:
                work_comp += -1
        work_comp *= count
        work += work_comp
    work = work/shots
    return work

In [52]:
#define system variables
N = 1 #number of qubits
Jz = 0.1 #ising interaction strength !!! Not used for 1-qubit
mu_x = 0.35 #transverse magnetic field strength
param_free_ham = Ising_Hamiltonian(N, Jz, [mu_x, 0, 0]) #parameter-free Hamiltonian
ising_ham0 = Ising_Hamiltonian(N, Jz, [mu_x, 0, 0]) #Hamiltonian at beginning of parameter sweep

#define simulation variables
tau = 10 #total trajectory time to evolve lambda from 0 to 1
dtau = 1.0 #time-step for trajectory
num_steps = int(tau/dtau)
T = 100 #total number of trajectories
dt =  dtau #timestep for Trotter approximation: setting equal to dtau means one trotter-step per time-step in evolution
lamba_protocol = np.linspace((dtau/tau),1.0,num_steps)
dldt = 0.1 # d(lambda)/d(tau)
shots = 1000

#define QITE variables
beta = 4.0 #inverse temperature of systems
dbeta = 0.5 #step-size in beta for QITE
domain = 1 #domain of opertators for QITE

In [54]:
#create program to move to x-basis for measurement
prog_xBasis = Program(N)
prog_xBasis.get_x_basis_prog()

#first state should be random
measured_metts_state = random_bitstring(N)
#subsequent entries are derived from running QMETTS on the previously derived
#state and measuring a random observable to get the state for the subsequent run

#need to sum work over each trajectory and then average over works
work = []
#loop over trajectories
for i in range(T):
    #print(i)
    psi0 = qite.get_state_from_string(measured_metts_state)
    prog_TS = Program(N)
    prog_TS.make_ps_prog(measured_metts_state)
    #print(measured_metts_state)
    #print("PS circuit")
    #prog_TS.print_list()
    prog_qite = Program(N)
    #note QITE algorithm should evolve state by beta/2 for temperature beta
    prog_qite.make_QITE_prog(ising_ham0, beta/2.0, dbeta, domain, np.asarray(psi0), 1.0)
    #print("QITE circuit")
    #prog_qite.print_list()
    prog_TS.append_program(prog_qite)
    #print("TS circuit")
    #prog_TS.print_list()
    #make and run qmetts program
    prog_qmetts = Program(N)
    prog_qmetts.append_program(prog_TS)
    #make random measurement operator
    if (i%2 == 0):
        prog_qmetts.append_program(prog_xBasis)
    #print("QMETTS circuit")
    #prog_qmetts.print_list()
    results = run_ibm(simulator, prog_qmetts,1)
    #update measured metts state for next trajectory
    measured_metts_state = results[0][0]
    #make and run JE program
    prog_JE = Program(N)
    prog_JE.append_program(prog_TS)
    #loop over time-steps in trajectory i
    work_i = 0    
    for step in range(num_steps):
        #print(step)
        #make Hamilton Evolution program for given time-step of given trajectory
        prog_hamEvol = Program(N)
        prog_hamEvol.make_hamEvol_prog(step, dtau, dt, lamba_protocol, param_free_ham)
        #complete JE program: combing IPS preparation, QITE, and Hamiltonian evolution
        prog_JE.append_program(prog_hamEvol)
        prog_JE.append_program(prog_xBasis)
        results = run_ibm(simulator, prog_JE, shots)
        #print(results)
        #print(get_work(results, shots))
        work_i += dldt*dtau*get_work(results, shots)
        work_i *= mu_x
    work.append(work_i)


print(work)

#FE = get_free_energy(work)  !!This function not yet implemented

[0.00014749874417865365, -0.0006405521653983574, 0.02908095370004399, 0.002338567013953381, 0.028656395802402947, 0.00025658580128508075, 0.0282037577100959, 0.00044623905656449475, -0.0011555305593725189, 0.0016570151113676577, -0.00016752392494195095, -7.515890796490239e-05, 0.000557159487064005, -0.0001905285439097834, 0.0009743182699777729, 0.000385988779241817, -0.0008550569402216398, 0.00226901620946082, -0.00036458514882534935, 3.409752945841837e-06, 0.0004877935458860366, 0.00023697634708439648, -0.00045543673376755646, 0.027766634216621695, 0.02986682357413248, -0.00021630348903269468, 0.028993168820946914, 0.0016875151372482827, 0.002093115010137812, 0.0005230290387427924, -0.00033928173404535063, 0.0014725132712438494, -0.0005998292456020683, 0.0005946153437697286, 0.002043495339504473, 0.0020902673974370536, 0.0017235625540794343, 0.0019518126247145323, 0.0009583589755799621, 7.759727944623221e-05, 0.00197452606872213, 0.028750048996541388, 0.0016440848723988286, 0.00109486